In [4]:
# %%capture
# !pip install selenium

## 지금까지 나온 문제
    대문 img를 말고 하단에 img가 추가 되어있는 책과 아닌책이 있어서 한칸씩 밀리는 경우
    책 속으로 라고하는 챕터가 있거나 없거나
    XPATH를 사용하지 않고 get_attribute('class')로 찾아야할지?
    ------

    추천사,책속으로,출판사서평,혹은 없는경우도 있음. div[10]
    
    * 형태 예제
        책소개 - 작가정보 - 목차 - 출판사서평
        책소개 - 작가정보(번역자까지 2개) - 목차 - 추천사 - 출판사서평
        책소개 - 작가정보 - 목차 - 책 속으로
        책소개 - 작가정보 - 목차 - 책 속으로 - 출판사 서평
        

In [14]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
import pandas as pd
import re
from bs4 import BeautifulSoup as Bs
import requests
import lxml.html

### request / bs

In [17]:
url = 'https://product.kyobobook.co.kr/detail/S000201478042'
r = requests.get(url)
r.status_code
bs = Bs(r.content,'html.parser')
bs


<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">

<meta charset="utf-8"/>
<html xmlns="http://www.w3.org/1999/xhtml">
<head>
<title> 서비스 점검 - 교보문고 </title>
<meta content="width=device-width, initial-scale=1.0, maximum-scale=1.0, minimum-scale=1.0, user-scalable=no, target-densitydpi=medium-dpi" name="viewport"/>
<meta content="" name="author">
<meta content="" name="keywords">
<meta content="" name="description"/>
<style type="text/css">
	* {padding:0;margin:0;list-style:none;}
  </style>
</meta></meta></head>
<body>
<style type="text/css">
	.errorPage_wrap {width:680px;margin:117px auto 0;}
	.errorPage_wrap .logo {font-size: 0; line-height: 0; text-align:center;}
	.errorPage_wrap .title {font-size:45px; color:#333; margin:23px 0 0 0; text-align:center;}
	.errorPage_wrap .title strong{color:#ff6600;}

	.errorPage_wrap .errorMsg01_wrap {margin:45px 0 46px;text-align:center; font-size:18px; line-height:1.4; color:#

In [12]:
bs.find_all('div',re.compile('product_detail_area.'))

[]

### selenium

In [53]:
driver = webdriver.Chrome()
# driver.get('https://product.kyobobook.co.kr/new/#?page=1&sort=new&year=2023&month=04&week=3&per=20&saleCmdtDvsnCode=KOR&gubun=newGubun&saleCmdtClstCode=')
# driver.get('https://product.kyobobook.co.kr/detail/S000201423559')
driver.get('https://product.kyobobook.co.kr/detail/S000201470540')
# driver.get('https://product.kyobobook.co.kr/detail/S000201478042')

In [54]:
for i in driver.find_elements(By.XPATH,'//*[@id="scrollSpyProdInfo"]/div'):
    print(i.text)


북카드
함께 구매한 책
나의 돈 많은 고등학교 친구
송희구
16,200원
엣지 한 끗의 차이를 만드는 내 안의 힘
로라 후앙. 이윤진
16,020원
숨 쉴 때마다 새로운 내가 된다면
마셜 골드스미스. 마크 라이터. 안솔비
17,100원
디깅(Digging)
박치은
16,200원
신뢰의 끈을 놓치지 말라
헤롤드 셔먼. 김경희
15,750원
일머리 문해력
송숙희
15,120원
키워드 Pick

기브
리원
정체
저항
무력
단계
더보기

책 소개
이 책이 속한 분야
국내도서 > 자기계발 > 성공/처세 > 자기관리/처세
구독자 32만 명의 유튜브 〈푸릉_렘군〉 대표, 대한민국 최고의 부동산 전문가,
10만 베스트셀러 작가, 3040 직장인들의 멘토,
렘군의 첫 자기계발서 출간!
어느덧 직장경력 10여 년, 항상 최선을 다해 열심히 살아온 것 같은데 생각보다 손에 쥐고 있는 것이 별로 없다고 느끼는 사람들이 꽤 많다. 자기계발에 소홀한 것도 아니고, 책도 꽤 읽고 공부도 한 것 같은데 왜 이렇게 회사를 벗어나서는 할 줄 아는 게 없다고 느껴지는 걸까? 지난 5년간 렘군은 ‘시작 컨설팅’이라는 이름으로 평범한 사람들이 자신의 강점을 찾고 제2의 인생 설계를 할 수 있도록 도왔다. 그는 이 과정을 통해 얻은 지식을 ‘아웃풋(output)’이라는 단어로 설명한다. 아웃풋은 단순히 어떤 생산물 또는 결과물만을 의미하지 않는다. 저자는 이를 타인을 위해, 세상과 연결되기 위해 제공하는 생산물로 정리한다.
우리는 늘 자신이 완벽하지 않다고 생각해 강의를 듣고, 책을 읽으며 인풋만 일삼는다. 남들이 만들어 놓은 지식, 콘텐츠, 물건 등을 소비하는 데 급급해 자신이 무엇을 세상에 제공할 수 있는지 잊고 산다. 그러나 조금 부족하더라도, 완벽하지 않더라도 자신의 아웃풋을 세상에 내놓는 바로 그 순간, ‘소비자의 영역’에서 벗어나 ‘생산자의 영역’으로 진입하게 된다. 이때 비로소 삶에 변화가 일어나는 것이다. 이렇게 저자는 ‘아웃풋 법칙’을 6단계로 정리해 사람들이 쉽고 간단하게 생산자

In [68]:
#책소개 
#위에 이미지가 있을경우 한칸씩 밀려서 작성된다?
print(driver.find_element(By.XPATH,'//*[@id="scrollSpyProdInfo"]/div[4]').get_attribute('class'))

product_detail_area detail_img


In [55]:
def img_finder():
    #product_detail_area detail_img / product_detail_area book_intro
    standard = driver.find_element(By.XPATH,'//*[@id="scrollSpyProdInfo"]/div[4]').get_attribute('class')
    if standard == 'product_detail_area detail_img':
        pass
    elif standard == 'product_detail_area book_intro':
        pass
    else:
        return print('예외처리가 안된 XPATH')
        


In [69]:
# title = driver.find_element(By.XPATH,'//*[@id="contents"]/div[1]/div/div[1]/div/div[1]/div[1]/div/h1/span').text
title = driver.find_element(By.CLASS_NAME,'prod_title').text
title

'한류 외전'

In [70]:
# sub_title = driver.find_element(By.XPATH,'//*[@id="contents"]/div[1]/div/div[1]/div/div[3]/div/div[1]/div/span').text
sub_title = driver.find_element(By.CLASS_NAME,'prod_desc').text
sub_title

'설계되지 않은 성공, K컬처산업의 운명을 바꾼 9가지 결정적 장면'

In [71]:
# author = driver.find_element(By.XPATH,'//*[@id="contents"]/div[1]/div/div[2]/div/div[1]/div/div[2]/div[1]/div/div/a').text
author = driver.find_element(By.CLASS_NAME,'author').text
# author = driver.find_element(By.XPATH,'//*[@id="contents"]/div[1]/div/div[2]/div/div[1]/div/div[1]/div[1]/div/div/a').text
# //*[@id="contents"]/div[1]/div/div[2]/div/div[1]/div/div[1]/div[1]/div/div/a img 있는 url
#//*[@id="contents"]/div[1]/div/div[2]/div/div[1]/div/div[2]/div[1]/div/div/a img 없는 url
author

'김윤지 저자(글)'

In [72]:
# price = driver.find_element(By.XPATH,'//*[@id="contents"]/div[1]/div/div[2]/div/div[3]/div[1]/div[2]/div').text
price = driver.find_element(By.CLASS_NAME,'prod_price_box').text
price

'10% 16,920원 18,800원'

In [73]:
img = driver.find_element(By.XPATH,'//*[@id="contents"]/div[1]/div/div[2]/div/div[2]/div[1]/div[1]/ul/li[1]/div/div[2]/img').get_attribute('src')
# img = driver.find_element(By.CLASS_NAME,'portrait_img_box portrait').get_attribute('src')
img

'https://contents.kyobobook.co.kr/sih/fit-in/458x0/pdt/9791167740977.jpg'

In [74]:
# cls = driver.find_element(By.XPATH,'//*[@id="scrollSpyProdInfo"]/div[4]/div[1]/ul/li/a[2]').text
cls = driver.find_elements(By.CLASS_NAME,'intro_category_link')[1].text
# cls = driver.find_element(By.XPATH,'//*[@id="scrollSpyProdInfo"]/div[5]/div[1]/ul/li/a[2]').text
# //*[@id="scrollSpyProdInfo"]/div[5]/div[1]/ul/li/a[2] img 있는 url
# //*[@id="scrollSpyProdInfo"]/div[4]/div[1]/ul/li/a[2] 없는 url
cls

'정치/사회'

In [75]:
# intro = driver.find_element(By.XPATH,'//*[@id="scrollSpyProdInfo"]/div[4]/div[2]/div').text
# intro = driver.find_element(By.XPATH,'//*[@id="scrollSpyProdInfo"]/div[5]/div[2]').text
intro = driver.find_element(By.CLASS_NAME,'intro_bottom').text
intro

'“우리가 잘한 걸까, 세계가 이상한 걸까?”\n세계를 놀라게 한 K컬처산업, 그 도약의 순간들에 관하여\n문화산업 연구자 김윤지가 정리한 30년 한류 막전막후\nBTS 멤버 지민의 빌보드 1위, 〈오징어 게임〉에서 〈더 글로리〉로 이어지는 넷플릭스 글로벌 1위 행진, 〈기생충〉과 윤여정의 오스카 수상을 보면 자연스럽게 이런 질문을 던지게 된다. “도대체 그동안 무슨 일이 일어난 걸까?” 거대한 외국자본과의 경쟁에서 살아남기 위해 발버둥 치던 변방의 문화산업이 세계를 강타하리란 걸 누가 알았을까? 할리우드 영화사들이 한국 영화시장에 직접 배급하는 걸 막기 위해 극장에 뱀을 풀던 1988년에 한국영화가 오스카를 휩쓰는 미래를 상상한 사람은 아무도 없었고, 지역 행사를 목표로 댄스음악을 만들던 1990년대에 빌보드 차트 진입은 꿈도 꿀 수 없는 일이었다. 그러나 상상보다 더 상상 같은 일들은 우리의 당연한 일상이 되었다. 친숙하지만 이해할 수 없는 이 현상을 이제는 명쾌하게 설명할 수 있어야 하지 않을까? 《한류 외전》은 영화시장 개방에서 BTS에 이르는 30여 년 동안 한국의 문화산업에 생긴 일들을 9개의 장면을 통해 설명한다.\n\n우리가 알지 못했던 K컬처산업의 역동적인 역사\n문화산업이 성공하기 위한 토양은 어떻게 만들어지는가?\n\nIMF 외환 위기가 없었다면 이를 극복하기 위한 새로운 산업 동력도 절실하지 않았을 것이고, 문화산업정책을 전폭적으로 입안하지 못했을 가능성이 높았다. 1980년대부터 이어져온 민주화 열기로 자유로운 창작 분위기를 조성하지 못했다면 우리 드라마, 영화의 수준도 계속 군부 시대의 틀 안에 놓였을 수도 있다. -‘프롤로그’ 중에서\n\n저자가 정리한 9개의 장면에는 한류와 연결하기 힘들었던 의외의 순간들이 존재한다. 아시아를 강타한 외환 위기와 한류드라마 사이에는 어떤 관계가 있을까? IMF 이후의 벤처캐피털은 박찬욱과 봉준호에게 어떤 기회를 주었을까? 위기에 몰린 가요 기획사는 왜 해외 진출을 선택할 수밖에 없었을까? 기회가 위

In [76]:
# contents = driver.find_element(By.XPATH,'//*[@id="scrollSpyProdInfo"]/div[8]/div[2]/div[1]/div/ul/li').text
# contents = driver.find_element(By.XPATH,'//*[@id="scrollSpyProdInfo"]/div[9]/div[2]/div[1]/div/ul/li').text
contents = driver.find_element(By.CLASS_NAME,'book_contents_item').text
contents


'프롤로그: K컬처산업의 토양은 무엇이었나?\n\n1장 시장개방이 만든 위기와 기회\n장면1 할리우드에 뱀으로 맞선 사람들\n세계시장의 빗장을 연 우루과이라운드 / 아무도 예상하지 못한 미래\n\n2장 〈쥬라기 공원〉에서 문화 융성까지, 새로운 산업 동력 찾기\n장면2 자동차 150만 대보다 강력한 공룡의 습격\n놀고먹는 일의 충격적인 경제효과 / 한류, 정부정책의 산물? 설계되지 않은 성공? / 불안한 세계화 시대, 딴따라에게서 미래를 찾다 / 팔길이 원칙, 지원은 하되 간섭하지 않는다 / 성장과 보호의 딜레마 / 국가의 도구가 된 한류\n\n3장 자유로운 드라마 산업이 만든 글로벌 공동체\n장면3 저렴하고 질 좋은 한국 드라마를 팝니다\n90년대 드라마의 질적 전환을 가져온 두 사건 / 대만이 한국 드라마 해외 진출의 최초 거점이 된 이유 / 드라마 산업화의 가능성을 제시한 〈겨울연가〉 / 아시아 시장을 넘어설 가능성을 보여준 〈대장금〉 / 초고속인터넷망의 비판적 시청자들 / 언어의 장벽을 뛰어넘는 한류 팬들 / 민주화, 자율성, 공정한 경쟁\n\n4장 낭만의 시대에서 투자의 시대로, 벤처 투자 대상이 된 영화\n장면4 지방 배급업자 김 사장이 한숨을 쉰 이유\n자본조달을 위한 유일한 선택지 / 삼성, LG, 대우, 영화시장에 진격한 대기업들 / IMF로 인한 1차 퇴각 / 새로운 영화 세대가 등장하다 / 문화산업 투자를 위해 총대를 멘 정부 / 영화로 눈을 돌린 투자자들 / 투자 촉진의 마중물이 된 영상전문투자조합 / 더 이상의 주먹구구는 없다, 금융계와 영화계의 신뢰 구축 / 한국영화 르네상스의 씨앗들\n\n5장 K팝 제조 시스템의 역동적인 시장 개척\n장면5 베이징 공인체육관을 불태운 H.O.T.\n신조어 ‘한류’는 어디에서 왔을까? / 빌보드 이전에 길보드가 있었다 / 10대 소비자를 위한 아이돌 제작 시스템 / 유통기간 7년의 아이돌 / MP3가 불러온 악몽 / 세계화 외에는 방법이 없다 / 난공불락의 미국 시장 / ‘미국식’으로 자리 잡고 싶었던

In [77]:
# inner_book = driver.find_element(By.XPATH,'//*[@id="scrollSpyProdInfo"]/div[10]/div[2]/div[1]/div/p').text
inner_book = driver.find_element(By.CLASS_NAME,'product_detail_area.book_inside').text
inner_book

'책 속으로\n전 세계가 거대한 하나의 시장으로 변화하는 치열하던 그 순간, 당황한 한국 영화인들은 영화관에 뱀을 풀었다. 콘크리트 바닥에 풀어놓은 뱀들이 힘을 쓰지 못하고 맥없이 죽는 모습이 말해 주듯 그게 정답이 아님은 뱀을 풀었던 영화인들도, 지켜보는 우리도 모두 알았다. 그런데 그런 강한 저항의 움직임은 정부도, 업계도 변화가 필요하다는 것을 감지하게 했다.(1장 시장개방이 만든 위기와 기회) -28~29쪽\n\n아마도 대기업이 든든히 시장에 버티고 있었다면 초기 시장 진출 과정은 조금 달랐을지도 모른다. 하지만 대기업이 없었기에 한국 드라마 산업은 누구나 성공할 수도, 누구나 실패할 수도 있는 공정한 경쟁의 장이 되었다. 대기업의 부재 덕분에 운동장은 평평했다. 향후 한류드라마 수준이 빠르게 향상된 데는 평평한 운동장에서 동등하게 경쟁하면서 키운 힘의 영향도 있었다.(3장 자유로운 드라마 산업이 만든 글로벌 공동체) -92쪽\n\nSM이 K팝 기획사로서 처음 코스닥에 입성한 것은 K팝 기획사가 정상적 수익모델을 가진 기업으로 인정받았다는 점에서 의의를 가진다. SM은 아이돌 육성과 해외 사업 확장을 위해 자본이 필요하다는 계획을 밝혔고, 시장이 이를 정상적 기업활동으로 인정했다는 의미기 때문이다. 즉 K팝 산업이 성장, 발전하기 위해 어떤 자본이 필요하고, 그런 활동을 하는 기업의 원형은 어떤 형태여야 하는지가 검증이 된 셈이다.(6장 엔터테인먼트도 안정적 산업이 될 수 있다) -193쪽\n\n음악적 진정성은 견고한 북미 음악시장의 장벽을 깨뜨리는 가장 중요한 비기였을까? 즉 과거의 K팝 아이돌도 음악적 진정성만 갖추었더라면 북미 시장의 벽을 뚫고 더 빨리 진입할 수 있었을지가 문제다. 꼭 그렇지는 않았을 것으로 보인다. K팝이 변화한 것만큼 미국 시장도 변화해 왔고, 그런 변화가 있었기에 BTS같은 아시아 아이돌에게도 기회가 생겼기 때문이다.(7장 팬덤이라는 세계화 전진 기지) -214쪽\n\n일본 대중문화 개방 방침이 준비되기 시작할 무렵 김대중 

In [79]:
recommend = driver.find_element(By.CLASS_NAME,'product_detail_area.book_recommend').text
recommend

'추천사\n정길화 (한국국제문화교류진흥원장, 전 MBC PD•중남미지사장)\n실증적인 자료와 검증, 때로는 감동적인 서사를 통해 한국 문화산업의 역동성을 구성한다. 《한류 외전》의 출현은 이제 본격적인 한류학이 탄생했음을 알려준다.\n조영신 (SK브로드밴드 경영전략그룹장)\n담담하고도 대담하게 한류의 탄생과 성장을 그린 대서사극이다. 미래에서 본 과거의 역사물이자 갈 길을 보여주는 SF물이다.\n심재명 (명필름 대표)\nK콘텐츠로 불리게 된 한국 대중문화산업, 그 역동적인 30년의 시간을 생생하게 살려낸 성실한 기록이자 산업의 역사를 이해하기에 손색없는 전문 서적이다.\n펼치기'

In [80]:
# publisher_review = driver.find_element(By.XPATH,'//*[@id="scrollSpyProdInfo"]/div[9]/div[2]/div[1]/div/p').text
# publisher_review = driver.find_element(By.XPATH,'//*[@id="scrollSpyProdInfo"]/div[11]/div[2]/div[1]/div/p').text
publisher_review = driver.find_element(By.CLASS_NAME,'product_detail_area.book_publish_review').text
publisher_review


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":".product_detail_area.book_publish_review"}
  (Session info: chrome=112.0.5615.137)
Stacktrace:
Backtrace:
	GetHandleVerifier [0x00D1DCE3+50899]
	(No symbol) [0x00CAE111]
	(No symbol) [0x00BB5588]
	(No symbol) [0x00BE08F9]
	(No symbol) [0x00BE0AFB]
	(No symbol) [0x00C0F902]
	(No symbol) [0x00BFB944]
	(No symbol) [0x00C0E01C]
	(No symbol) [0x00BFB6F6]
	(No symbol) [0x00BD7708]
	(No symbol) [0x00BD886D]
	GetHandleVerifier [0x00F83EAE+2566302]
	GetHandleVerifier [0x00FB92B1+2784417]
	GetHandleVerifier [0x00FB327C+2759788]
	GetHandleVerifier [0x00DB5740+672048]
	(No symbol) [0x00CB8872]
	(No symbol) [0x00CB41C8]
	(No symbol) [0x00CB42AB]
	(No symbol) [0x00CA71B7]
	BaseThreadInitThunk [0x763A0099+25]
	RtlGetAppContainerNamedObjectPath [0x77107B6E+286]
	RtlGetAppContainerNamedObjectPath [0x77107B3E+238]


In [81]:
# ISBN = driver.find_element(By.XPATH,'//*[@id="scrollSpyProdInfo"]/div[10]/div[2]/table/tbody/tr[1]/td').text
# ISBN = driver.find_element(By.XPATH,'//*[@id="scrollSpyProdInfo"]/div[12]/div[2]/table/tbody/tr[1]/td').text
ISBN = driver.find_element(By.CLASS_NAME,'tbl_row').find_elements(By.TAG_NAME,'td')[0].text
ISBN

'9791167740977'

### 책마다 XPATH가 달라요..

In [5]:
driver.get('https://product.kyobobook.co.kr/detail/S000201470540')

<selenium.webdriver.remote.webelement.WebElement (session="3787e6e269ddb79403397ca97154aa80", element="b209b8aa-b541-495b-ae08-b0871f515eda")>

In [13]:
driver.find_element(By.XPATH,'//*[@id="scrollSpyProdInfo"]/div[5]/div[2]').text


'구독자 32만 명의 유튜브 〈푸릉_렘군〉 대표, 대한민국 최고의 부동산 전문가,\n10만 베스트셀러 작가, 3040 직장인들의 멘토,\n렘군의 첫 자기계발서 출간!\n어느덧 직장경력 10여 년, 항상 최선을 다해 열심히 살아온 것 같은데 생각보다 손에 쥐고 있는 것이 별로 없다고 느끼는 사람들이 꽤 많다. 자기계발에 소홀한 것도 아니고, 책도 꽤 읽고 공부도 한 것 같은데 왜 이렇게 회사를 벗어나서는 할 줄 아는 게 없다고 느껴지는 걸까? 지난 5년간 렘군은 ‘시작 컨설팅’이라는 이름으로 평범한 사람들이 자신의 강점을 찾고 제2의 인생 설계를 할 수 있도록 도왔다. 그는 이 과정을 통해 얻은 지식을 ‘아웃풋(output)’이라는 단어로 설명한다. 아웃풋은 단순히 어떤 생산물 또는 결과물만을 의미하지 않는다. 저자는 이를 타인을 위해, 세상과 연결되기 위해 제공하는 생산물로 정리한다.\n우리는 늘 자신이 완벽하지 않다고 생각해 강의를 듣고, 책을 읽으며 인풋만 일삼는다. 남들이 만들어 놓은 지식, 콘텐츠, 물건 등을 소비하는 데 급급해 자신이 무엇을 세상에 제공할 수 있는지 잊고 산다. 그러나 조금 부족하더라도, 완벽하지 않더라도 자신의 아웃풋을 세상에 내놓는 바로 그 순간, ‘소비자의 영역’에서 벗어나 ‘생산자의 영역’으로 진입하게 된다. 이때 비로소 삶에 변화가 일어나는 것이다. 이렇게 저자는 ‘아웃풋 법칙’을 6단계로 정리해 사람들이 쉽고 간단하게 생산자의 삶을 살 수 있도록 안내한다.\n\n아웃풋 법칙 1단계. 피라미드 밖으로 뛰쳐나가라.\n아웃풋 법칙 2단계. 성공의 사분면을 찾아라.\n아웃풋 법칙 3단계. 나만의 정체성을 발견하라.\n아웃풋 법칙 4단계. 세상을 향해 아웃풋하라.\n아웃풋 법칙 5단계. 넘버원이 아닌 온리원이 돼라.\n아웃풋 법칙 6단계. 저항을 완전히 무력화시켜라.\n\n친절하면서도 예리한 저자의 관점을 즐기면서 차근차근 실행하다 보면, 어느새 나만의 아웃풋을 만들어내는 자신을 발견할 수 있을 것이다. 이 일은 지금까지 습관과 관성

In [23]:
print(driver.find_element(By.XPATH,'//*[@id="scrollSpyProdInfo"]/div[4]').get_attribute('class'))

product_detail_area detail_img


In [30]:
driver.get('https://product.kyobobook.co.kr/detail/S000201505566')

In [31]:
print(driver.find_element(By.XPATH,'//*[@id="scrollSpyProdInfo"]/div[4]').get_attribute('class'))

product_detail_area book_intro
